In [8]:
import findspark
findspark.init('/opt/spark')  # Adjust the path if Spark is installed elsewhere

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("example").getOrCreate()

In [10]:
# Define the path to your JSON file
from pyspark.sql.functions import col, udf, to_date, when, regexp_extract, explode, col, regexp_replace, to_timestamp, concat, lit, array, trim
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType, ArrayType
import json
import re
import pymongo
json_file_path = 'vieclam24h.json'
# Read the JSON file into a DataFrame
df = spark.read.option("multiline","true").json(json_file_path)

# df_exploded = df.select(explode(col("location")).alias("unique_category"))

# unique_categories = df_exploded.select("unique_category").distinct().collect()
# print(len(unique_categories))
# for row in unique_categories:
#     pass
#     print(row.unique_category)
# unique_experience_df = df.select("location").distinct()

# unique_experience_df.show()

# from pyspark.sql.functions import col
unique_values = df.select(col("sex")).distinct().collect()

# In ra các giá trị duy nhất
for row in unique_values:
    if row.sex is None:
        print("None ne")
    print(row.sex)


# print(df.size)


#rdd.toDF().select("salary").show(1)

# Salary


Nữ
Nam
None ne
None


In [11]:
# Salary
def parse_salary(salary):
    if salary is None:
        return {"type": 0 }
    elif (salary.strip().lower() == "thoả thuận" or salary.strip().lower() == "thỏa thuận"):
        return {"type": 3 }
    elif "triệu" in salary:
        salary = salary.replace(" triệu", "").replace(",", ".").strip()
        if "-" in salary:
            min_salary, max_salary = salary.split("-")
            return {"type": 1, "min": float(min_salary), "max": float(max_salary) }
        else:
            return {"type": 2, "fixed_value": float(salary)}
# Age
def parse_age(age):
    if age is None:
        return { "type": 0 }
    age = age.strip().lower()
    if "tuổi" in age:
        age = age.replace(" tuổi", "").strip()
        if "-" in age:
            min_age, max_age = age.split("-")
            return { "type": 1, "min": int(min_age), "max": int(max_age) }
        else:
            return { "type": 2, "value": int(age) }
    return { "type": 0 }

# Experience
def parse_experience(experience):
    if experience is None or experience == "Chưa có kinh nghiệm":
        return { "type": 0 }
    if "Dưới" in experience:
        strip_experience = experience.replace("Dưới", "").replace("năm", "").strip()
        max = strip_experience
        return { "type": 3, "max": int(max) }
    elif "Hơn" in experience:
        strip_experience = experience.replace("Hơn", "").replace("năm", "").strip()
        min = strip_experience
        return { "type": 4, "min": int(min) }
    else:
        fixed = experience.replace("năm", "").strip()
        return { "type": 2, "fixed": int(fixed)}
    return { "type": 0 }
    
# Type
df = df.withColumn(
    "type",
     when(col("type") == "Toàn thời gian cố định", array(lit(0)))
    .when(col("type") == "Bán thời gian cố định", array(lit(2)))
    .when(col("type") == "Toàn thời gian tạm thời", array(lit(1)))
    .when(col("type") == "Khác", array(lit(5)))
    .otherwise(array(lit(0)))
)

# Sex
df = df.withColumn(
    "sex",
     when(col("sex") == "Nam", 1)
    .when(col("sex") == "Nữ", 2)
    .otherwise(0)
)

# Expiration
df = df.withColumn("expiration", to_timestamp(concat(col("expiration"), lit(" 00:00:00")), "dd/MM/yyyy HH:mm:ss"))

# Update at
df = df.withColumn("update_time", to_timestamp(concat(col("update_time"), lit(" 00:00:00")), "dd/MM/yyyy HH:mm:ss"))

# Description
def clean_description(description):
    return [re.sub(r'^[\-\•\d\.\s]+', '', desc).strip() for desc in description]
clean_description_udf = udf(clean_description, ArrayType(StringType()))
df = df.withColumn("description", clean_description_udf(col("description")))

# Role
def classify_role(role):
    if role == "Chuyên viên- nhân viên":
        return 0
    elif role == "Quản lý cấp cao" or role == "Quản lý cấp trung":
        return 1
    elif role == "Cộng tác viên":
        return 6
    elif role == "Quản lý nhóm- giám sát":
        return 5
    elif role == "Chuyên gia":
        return 7
    else:
        return 0
classify_role_udf = udf(classify_role, IntegerType())
df = df.withColumn("role", classify_role_udf(col("role")))

def normalize_benefit(benefits):
    return list(map(lambda benefit: re.sub(r'^[*\-+]+', '', benefit).strip(), benefits))

benefit_udf = udf(normalize_benefit, ArrayType(StringType()))
df = df.withColumn("benefit", benefit_udf(col("benefit")))
df = df.withColumn("requirement", benefit_udf(col("requirement")))

# location
def normalize_location(locations):
    arr = []
    for location in locations:
        parts = [part.strip() for part in location.split(',') if part.strip()]
        if len(parts) >= 2:
            province = parts[-1]
            district = parts[-2]
            address = ', '.join(parts[:-2])
            arr.append({"province": province, "district": district, "address": address})
        elif len(parts) == 1:
            arr.append ({"province": parts[-1], "district": None, "address": None })
        else: 
            arr.append({"address": None, "district": None, "province": None})
    return arr
    
rdd = df.rdd.map(lambda row: {
    **row.asDict(),
    "salary": parse_salary(row["salary"]),
    "age": parse_age(row["age"]),
    "location": normalize_location(row["location"]),
    "experience": parse_experience(row["experience"])
})


def insert_into_mongodb(partition):
    client = pymongo.MongoClient("mongodb://localhost:27017/", username='admin', password='20194856')
    db = client["thesis"]
    collection = db["jobs"]

    for record in partition:
        collection.insert_one(record)

# Insert dữ liệu vào collection "jobs"
rdd.foreachPartition(insert_into_mongodb)